# Análise de Sentimento

### Vinícius Nascimento Silva - 7557626

#### Bibliotecas utilizadas

Neste trabalho foi utilizada a biblioteca *NLTK* para o processamento do texto e a biblioteca *Sklearn* como fornecedora de algorítmos de aprendizado.

Outras bibliotecas utilizadas foram a *Panda* para a manipulação de arquivos csv e a numpy para a manipulação de dados.

In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer
import pyprind
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from datetime import datetime

### Processamento dos Tweets

A estratégia utilizada para criar a *Bag of Words* foi extrair as informações dos tweets foi tentar relacionar palavras com mesma raíz. Para isso foi utilizada técnicas de **stemização** e **lematização**. A lematização usa um dicionário para determinar qual é a raiz da palavra, mas não consegue diferenciar substantivos de verbos e a tokenização necessária para esse preprocessamento é muito lenta, então a stemização ajuda a reduzir as palavras não tratadas pela lematização.

Além as raízes das palavras, as pontuações '!' e '?' também foram consideradas.

In [2]:
vectorizer = CountVectorizer(analyzer = "word", max_features = 2500 ,tokenizer = None,preprocessor = None,stop_words = None) 
stemmer = LancasterStemmer()
lemmatizer = WordNetLemmatizer()


def reduce(words):
    words = map(lemmatizer.lemmatize, words)
    words = map(stemmer.stem, words)
    return words

def tweets_to_words( raw_review ):    
    
    #Remove caracteres indesejados
    letters_only = re.sub("[^a-zA-Z!?]", " ", raw_review) 
    #Separa 'Camel Case'
    separated = re.sub("([a-z])([A-Z])","\g<1> \g<2>",letters_only)
    #Separa pontuação
    separated = re.sub("([a-zA-Z])([?!])","\g<1> \g<2>",separated)
    separated = re.sub("([?!])([?!])","\g<1> \g<2>",separated)
    
    #Quebra palavras e deixa tudo minúsculo
    words = separated.lower().split()
    
    #Tira palavras sem sgnificados
    stops = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stops]
    
    #Reduz palavras para as suas raízes
    reduced_words = reduce(meaningful_words)
    
    return(" ".join(reduced_words))

def clear_tweets(tweets, verbose = False):
    n_tweets = tweets.size
    clean_tweets = []
    if verbose : bar = pyprind.ProgBar(n_tweets)
    for i in range( 0, n_tweets ):
        clean_tweets.append(tweets_to_words(tweets[i]))
        if verbose : bar.update()
    return clean_tweets

### Outras Features

A compania aérea e a quantidade de retweets também são features e foram implementadas de forma direta.

Uma questão interessante seria se o horário do tweet é um indicativo de sentimento. Para isso avaliamos o horário como a distância para o meio dia, ou seja, caso o tweet tenha sido publicado às 16h, o valor da feature é 4, cas o tenha sido ás 3 da manhã, o valor é 9. A ideia foi manter horários próximos com valores próximos e evitar "quebras" arbitrárias.

Uma aparente melhora no aprendizado pode ser observada, mas nada conclusivo:

Algorítmo | Sem feature de tempo | Centralizado às 12h | Centralizado às 6h
:---: | :---: | :---: | :---:
Perceptron | 85,43% | 88,52% | 89,05%
Linear Regression | 81,38% | 81,43% | 81,43%
Logistic Regression | 90,06% | 90,21% | 90,11%
Random Forest | 87,55% | 88,23% | 87,94%

Talvez, com mais dados, possamos indentificar se há realmente uma influência do horário.

In [3]:
vectorizerAirline = CountVectorizer(analyzer = "word",tokenizer = None,preprocessor = None,stop_words = None)

def clean_airline_name(airlines) :
    cleaned_names = []
    for airline_name in airlines :
        cleaned_names.append(re.sub(" ", "", airline_name))
    return cleaned_names

def generate_time_features(time_vector) :
    distances = []
    for string_date in time_vector :
        hour_of_the_day = datetime.strptime(string_date, "%Y-%m-%d %H:%M:%S %z").hour
        distance_from_midday = abs(12 - hour_of_the_day)
        distances.append(distance_from_midday)
    return np.matrix(distances).transpose()

In [4]:
def get_features(data, verbose = False, train = False) :
    clean_tweets = clear_tweets(data['text'], verbose)
    airlines = clean_airline_name(data['airline'])
    
    if train :
        word_features = vectorizer.fit_transform(clean_tweets).toarray()
        airlines_features = vectorizerAirline.fit_transform(airlines).toarray()
    else :
        word_features = vectorizer.transform(clean_tweets).toarray()
        airlines_features = vectorizerAirline.transform(airlines).toarray()
    
    time_features = generate_time_features(data['tweet_created'])
    retweets = np.matrix(data["retweet_count"])
    
    return np.hstack((word_features, retweets.transpose(), airlines_features, time_features))


### Algorítmos

Para decidir que algorítmo será utilizado para enviar os dados para o Kaggle separei 2000 tweets marcados para testar o erro fora da amostra. Os algorítmos escolhidos foram os vistos em aula (com a excessão de redes neurais) e o algorítmo de floresta aleatória utilizado no tutorial do próprio kaggle. O *Logistic Regression* teve a melhor performance na validação com um pouco menos de 10% de erro.

In [5]:
train = pd.read_csv("treinamento_parte.csv", header=0,delimiter=",")
train_data_features = get_features(train, verbose=True, train=True)
print(train_data_features.shape)

0%                          100%
[##############################] | ETA: 00:00:00

(5999, 2508)



Total time elapsed: 00:00:11


In [6]:
forest = RandomForestClassifier(n_estimators = 100) 
forest = forest.fit( train_data_features, train["airline_sentiment"])

In [7]:
perceptron = Perceptron(n_iter=100)
perceptron = perceptron.fit(train_data_features, train["airline_sentiment"])

In [8]:
regression = LinearRegression()
regression = regression.fit(train_data_features, train["airline_sentiment"])

In [9]:
logistic = LogisticRegression()
logistic = logistic.fit(train_data_features, train["airline_sentiment"])

In [10]:
validation = pd.read_csv("validacao.csv", header=0,delimiter=",")
validation_data_features = get_features(validation, verbose=True, train=False)
print(validation_data_features.shape)

0%                          100%
[##############################] | ETA: 00:00:00

(2073, 2508)



Total time elapsed: 00:00:02


In [11]:
perceptron_result = perceptron.predict(validation_data_features)

regression_result = regression.predict(validation_data_features)
regression_result = list(map(lambda x: x/abs(x),regression_result))

forest_result = forest.predict(validation_data_features)

logistic_result = logistic.predict(validation_data_features)

real_results = validation["airline_sentiment"]


def get_performance(test_results) :
    performance = 0.0
    for test, real in zip(test_results, real_results) :
        if test == real :
            performance += 1
    return performance/real_results.size
    

print ("Perceptron: ", get_performance(perceptron_result))
print ("LinearRegression: ", get_performance(regression_result))
print ("LogisticRegression: ", get_performance(logistic_result))
print ("Random Forest: ", get_performance(forest_result))

Perceptron:  0.8895320791123975
LinearRegression:  0.8678244090689822
LogisticRegression:  0.9073806078147613
Random Forest:  0.8876025084418717


### Kaggle

E este foi o código utilizado para fazer a submissão no kaggle:

In [13]:
final_train = pd.read_csv("treinamento.csv", header=0,delimiter=",")
final_train_features = get_features(final_train, verbose=True, train=True)

logistic = logistic.fit(final_train_features, final_train['airline_sentiment'])

final_test = pd.read_csv("teste-sem-resposta.csv", header=0,delimiter=",")
final_test_features = get_features(final_test, verbose=True, train=False)

logistic_result = logistic.predict(final_test_features)

output = pd.DataFrame(\
    data={"id":final_test["id"], \
          "airline_sentiment":logistic_result})

output.to_csv( "resultado.csv", index=False )

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:09
0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:04
